In [1]:
#!pip install implicit

In [2]:
#!pip install implicit==0.4.8 --no-use-pep517

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [30]:
test_size_weeks = 80

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [42]:
data_train.tail(20)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
154052,88,28154672658,96,883404,1,0.99,404,0.00,1918,14,0.0,0.0
154053,88,28154672658,96,888104,1,4.99,404,0.00,1918,14,0.0,0.0
154054,88,28154672658,96,893018,1,1.97,404,-1.02,1918,14,0.0,0.0
154055,88,28154672658,96,952981,1,0.50,404,-0.49,1918,14,0.0,0.0
154056,88,28154672658,96,960942,1,1.00,404,-1.69,1918,14,0.0,0.0
154057,88,28154672658,96,981760,1,0.89,404,-0.50,1918,14,0.0,0.0
154058,88,28154672658,96,1010308,1,3.99,404,0.00,1918,14,0.0,0.0
154059,88,28154672658,96,1021248,1,2.19,404,0.00,1918,14,0.0,0.0
154060,88,28154672658,96,1035243,1,1.79,404,-0.20,1918,14,0.0,0.0
154061,88,28154672658,96,1044078,1,2.00,404,-1.07,1918,14,0.0,0.0


In [39]:
data_train['sales_value']

0          1.39
1          0.82
2          0.99
3          1.21
4          1.50
          ...  
154067     1.98
154068     1.00
154069     1.00
154070     4.98
154308    35.32
Name: sales_value, Length: 150074, dtype: float64

In [37]:
data_train.shape[0]

150074

In [40]:
data_test.tail(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0
2396803,1167,41656790510,663,6430664,1,8.00,3385,0.00,1059,95,0.0,0.0


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [43]:
import pandas as pd
import math

def get_items_weights(data_train):
    weight=[]
    # получим список уникальных товаров
    item_id=data_train.item_id.unique()
    items_weights=pd.DataFrame()
    items_weights['item_id']=item_id
    weight=np.zeros(len(item_id), dtype=float)
    items_weights['weight']=weight
#     print(len(item_id))
    item_id=item_id.tolist()
    weight=weight.tolist()
#    print(type(weight))
#     print(len(weight))
#     print(data_train.shape[0])
    big_sales=0.0
    
    index = data_train.index
    for i in index:
        if (i%10000==0):
            print(i)
        sales=data_train.sales_value[i]
        quantity=data_train.quantity[i]
        sales_sum=sales*quantity
        ind=data_train.item_id[i]
        items_weights.weight[item_id.index(data_train.item_id[i])]+=sales_sum
       # print(items_weights.weight[item_id.index(data_train.item_id[i])])
        big_sales+=sales_sum
    print('big_sales=',big_sales) 
    
    # нормировка по весам сделаем, относительная доля
    items_weights['weight'] = items_weights['weight'] * (1 / big_sales)
        
    return items_weights

#data_train['sales_value']*data_train['quantity']
#weight=log(sales_sum товара)
df=get_items_weights(data_train)
df.tail(3)

0


<ipython-input-43-188208860ed9>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_weights.weight[item_id.index(data_train.item_id[i])]+=sales_sum


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
big_sales= 222829182.83001763


,item_id,weight
27509,8091167,4.442865e-09
27510,824322,8.392079e-09
27511,1072353,1.570710e-08


In [44]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекомендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    
    #items = np.array(items)
    #recs = np.random.choice(items, size=n, replace=False)
    recs = np.random.choice(items_weights['item_id'], n, p=items_weights['weight'])

    
    return recs.tolist()

In [48]:
%%time
# your_code
items = data_train.item_id.unique()
result=pd.DataFrame()
result['user_id']=data_train['user_id']
result['random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(df, n=5))
result.head(2)


KeyboardInterrupt: 

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [197]:
def precision_at_k(recommended_list, bought_list, k=5):
    print(len(recommended_list))   
    size_len=0
    flags_value=0
    precision=0.0
    ls=len(recommended_list)
    for i in range(0,ls):
        bought_list_t = np.array(bought_list[i])
        recommended_list_t = np.array(recommended_list[i])
    
#         bought_list = bought_list  # Тут нет [:k] !!
#         recommended_list_t = recommended_list_t[:k]
        print((bought_list_t.tolist()))
        
        print((recommended_list_t.tolist()))
        flags =  np.isin(bought_list_t.tolist(), recommended_list_t.tolist())
        print(flags)
        flags_value=flags_value+flags.sum()
        size_len=size_len+k
        print('flags=',flags_value)
        print('size_len=',size_len)
    
    print('size_len=',size_len)
    precision = flags_value / size_len

    return precision

In [198]:
import src

result = pd.read_csv('predictions/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [199]:
# random_recommendation
print(precision_at_k(result['random_recommendation'], result['own_purchases'], k=5))


2042
[999999, 1082185, 1029743, 995785, 1004906]
[5586238, 1015228, 866118, 2416733, 2603573]
False
flags= 0
size_len= 5
[999999, 1082185, 1098066, 6534178, 1127831]
[161354, 63027, 1027802, 12263694, 307395]
False
flags= 0
size_len= 10
[999999, 1082185, 1029743, 6534178, 1127831]
[13416054, 936084, 7410040, 9527114, 377218]
False
flags= 0
size_len= 15
[999999, 1082185, 1029743, 1127831, 995785]
[5574336, 990072, 868548, 995880, 842226]
False
flags= 0
size_len= 20
[999999, 1082185, 1029743, 1098066, 6534178]
[1277401, 94446, 3133282, 1925252, 855699]
False
flags= 0
size_len= 25
[999999, 1082185, 1029743, 1098066, 1127831]
[966310, 1114417, 896976, 15717035, 1054783]
False
flags= 0
size_len= 30
[999999, 1082185, 1029743, 6534178, 1127831]
[911088, 13133787, 10311511, 59994, 15596516]
False
flags= 0
size_len= 35
[999999, 1029743, 1098066, 6534178, 1127831]
[1038153, 1119575, 1018818, 6039624, 12487492]
False
flags= 0
size_len= 40
[999999, 1082185, 1029743, 1127831, 995785]
[1014751, 1045

size_len= 1075
[999999, 6534178, 1106523, 1134522, 1022872]
[1002852, 12605163, 943530, 9639587, 1007563]
False
flags= 0
size_len= 1080
[999999, 1082185, 1004906, 1068719, 1133018]
[9487187, 996754, 2443057, 1923849, 8068632]
False
flags= 0
size_len= 1085
[999999, 1068719, 1005186, 866227, 1058997]
[1048859, 997088, 837448, 192595, 9671846]
False
flags= 0
size_len= 1090
[999999, 1082185, 1029743, 1098066, 1004906]
[2929806, 9858936, 1001266, 8203610, 1532888]
False
flags= 0
size_len= 1095
[999999, 1082185, 1098066, 6534178, 1127831]
[1013953, 986684, 1599069, 9297364, 13038826]
False
flags= 0
size_len= 1100
[999999, 1029743, 1098066, 6534178, 1127831]
[1822362, 10215338, 1016495, 1716533, 2059258]
False
flags= 0
size_len= 1105
[999999, 1082185, 1029743, 1098066, 1044078]
[1337734, 13189652, 1008824, 9803278, 8064612]
False
flags= 0
size_len= 1110
[999999, 1098066, 995785, 1044078, 1033142]
[864225, 821863, 12695754, 7126079, 979783]
False
flags= 0
size_len= 1115
[999999, 1082185, 10297

[999999, 1082185, 6534178, 1127831, 995785]
[591677, 582065, 9445677, 10144306, 836922]
False
flags= 0
size_len= 2150
[999999, 1029743, 1098066, 6534178, 1004906]
[878337, 1090471, 1022675, 5592503, 1024206]
False
flags= 0
size_len= 2155
[999999, 1082185, 1029743, 1098066, 6534178]
[1107382, 1008531, 990588, 2804527, 15972878]
False
flags= 0
size_len= 2160
[999999, 1092026, 8090537, 8090521, 1102959]
[1440974, 438172, 405930, 6039621, 944334]
False
flags= 0
size_len= 2165
[999999, 1082185, 1098066, 6534178, 1127831]
[8086244, 9655600, 9368192, 1408520, 1042858]
False
flags= 0
size_len= 2170
[999999, 1082185, 6534178, 1127831, 995785]
[15831380, 9296911, 12757314, 925394, 1086384]
False
flags= 0
size_len= 2175
[999999, 1082185, 1029743, 6534178, 995785]
[1124860, 1059473, 12384737, 1035690, 1134860]
False
flags= 0
size_len= 2180
[999999, 1082185, 1029743, 1127831, 1004906]
[1001369, 6533297, 1135681, 901840, 1755075]
False
flags= 0
size_len= 2185
[999999, 1082185, 1029743, 1004906, 1044

[1069103, 1137467, 1029688, 315827, 562396]
False
flags= 0
size_len= 3220
[999999, 1082185, 1098066, 995785, 1004906]
[1010530, 1240605, 14025718, 7024853, 9297106]
False
flags= 0
size_len= 3225
[999999, 1082185, 1029743, 1098066, 1127831]
[10121823, 851502, 873345, 839112, 828295]
False
flags= 0
size_len= 3230
[999999, 1082185, 1029743, 6534178, 995785]
[276370, 887691, 16809107, 13777406, 1082377]
False
flags= 0
size_len= 3235
[999999, 1082185, 1029743, 1098066, 6534178]
[977999, 920769, 2596756, 12131542, 13842101]
False
flags= 0
size_len= 3240
[999999, 1082185, 1029743, 6534178, 1127831]
[6961770, 1014099, 17105032, 6919647, 8352817]
False
flags= 0
size_len= 3245
[999999, 1082185, 1029743, 1098066, 6534178]
[1175798, 1225488, 8020232, 988211, 15716203]
False
flags= 0
size_len= 3250
[999999, 1029743, 1096036, 1005186, 1007195]
[13381364, 1053241, 880139, 8068322, 15553360]
False
flags= 0
size_len= 3255
[999999, 1082185, 1029743, 1098066, 1044078]
[951284, 1018521, 8293280, 13095040,

flags= 0
size_len= 4290
[999999, 1082185, 1029743, 995785, 1004906]
[1000621, 3937837, 7160639, 1046135, 1127338]
False
flags= 0
size_len= 4295
[999999, 1082185, 1029743, 1098066, 995785]
[8068955, 995554, 10462537, 1019722, 1702149]
False
flags= 0
size_len= 4300
[999999, 1082185, 1029743, 6534178, 995785]
[3785499, 1094014, 1928262, 9859161, 6423954]
False
flags= 0
size_len= 4305
[999999, 1029743, 995785, 1004906, 1068719]
[5569230, 6463907, 1017235, 847294, 1861418]
False
flags= 0
size_len= 4310
[999999, 1082185, 1029743, 1133018, 5569230]
[12524734, 837107, 1078383, 1006542, 995811]
False
flags= 0
size_len= 4315
[999999, 1082185, 1029743, 1098066, 6534178]
[194638, 93320, 1023748, 979423, 1107030]
False
flags= 0
size_len= 4320
[999999, 1082185, 1029743, 1098066, 1127831]
[1111332, 10264156, 431392, 12301446, 919558]
False
flags= 0
size_len= 4325
[999999, 1082185, 6534178, 1004906, 1081177]
[1344645, 5571023, 1424069, 1110603, 963354]
False
flags= 0
size_len= 4330
[999999, 1082185, 6

flags= 0
size_len= 5360
[999999, 1082185, 1029743, 1098066, 1127831]
[968415, 1075793, 12488030, 9835899, 837700]
False
flags= 0
size_len= 5365
[999999, 1082185, 1098066, 1127831, 995785]
[10285000, 1043279, 41976, 835476, 7169088]
False
flags= 0
size_len= 5370
[999999, 1082185, 1029743, 1098066, 6534178]
[1014706, 834129, 44039, 870909, 886487]
False
flags= 0
size_len= 5375
[999999, 1082185, 1029743, 1098066, 6534178]
[899655, 3898854, 1065129, 169362, 1090267]
False
flags= 0
size_len= 5380
[999999, 1082185, 1098066, 6534178, 1127831]
[13008371, 857841, 13190071, 699449, 902815]
False
flags= 0
size_len= 5385
[999999, 1082185, 1098066, 6534178, 995785]
[5657557, 1136605, 1558546, 1028742, 10356086]
False
flags= 0
size_len= 5390
[999999, 1082185, 1029743, 1098066, 1127831]
[1091651, 5995423, 849019, 151446, 819423]
False
flags= 0
size_len= 5395
[999999, 1082185, 6534178, 995785, 1096036]
[846855, 923692, 1044300, 934618, 1022086]
False
flags= 0
size_len= 5400
[999999, 1082185, 1029743, 

size_len= 6430
[999999, 1082185, 1029743, 1098066, 6534178]
[354043, 8068463, 12325186, 12517932, 6553358]
False
flags= 0
size_len= 6435
[999999, 1082185, 1029743, 1098066, 1127831]
[13158058, 7124838, 7155759, 1119112, 5567233]
False
flags= 0
size_len= 6440
[999999, 1082185, 1029743, 6534178, 1127831]
[1705749, 12300965, 9879426, 13777086, 1090330]
False
flags= 0
size_len= 6445
[999999, 6534178, 995785, 5569230, 1096036]
[967705, 1110688, 7167030, 6944753, 9527559]
False
flags= 0
size_len= 6450
[999999, 1082185, 1029743, 1098066, 6534178]
[13987345, 9676901, 12674377, 1030224, 59878]
False
flags= 0
size_len= 6455
[999999, 1082185, 6534178, 1068719, 1044078]
[869604, 5565320, 9881463, 998868, 1129507]
False
flags= 0
size_len= 6460
[999999, 1082185, 1098066, 6534178, 1081177]
[1116766, 1003749, 1026053, 12306082, 8203606]
False
flags= 0
size_len= 6465
[999999, 1082185, 1004906, 1044078, 1076056]
[6555328, 959926, 1042684, 10456049, 1130917]
False
flags= 0
size_len= 6470
[999999, 1082185

[999999, 1082185, 1098066, 995785, 1004906]
[950289, 363658, 830887, 1009884, 10311269]
False
flags= 0
size_len= 7505
[999999, 1082185, 1029743, 1098066, 1081177]
[1276374, 8154479, 1124352, 10121752, 9803609]
False
flags= 0
size_len= 7510
[999999, 1082185, 1098066, 6534178, 1127831]
[13511223, 886655, 1232342, 15742000, 1147613]
False
flags= 0
size_len= 7515
[999999, 1082185, 1029743, 1098066, 1127831]
[860487, 7111701, 10311773, 868386, 5772053]
False
flags= 0
size_len= 7520
[999999, 1082185, 1029743, 6534178, 1127831]
[43759, 1099975, 9194532, 920725, 10255421]
False
flags= 0
size_len= 7525
[999999, 1082185, 1098066, 6534178, 995785]
[986006, 903895, 5571859, 1028243, 1601122]
False
flags= 0
size_len= 7530
[999999, 1082185, 1029743, 1098066, 1127831]
[13377247, 6391534, 9802714, 8181569, 1132693]
False
flags= 0
size_len= 7535
[999999, 1082185, 1098066, 6534178, 1127831]
[1092858, 943316, 1205332, 385452, 13955331]
False
flags= 0
size_len= 7540
[999999, 1082185, 1098066, 1127831, 995

size_len= 8650
[999999, 1082185, 6534178, 1127831, 995785]
[1619910, 1077008, 12599980, 13777373, 2412607]
False
flags= 0
size_len= 8655
[999999, 1082185, 1029743, 1098066, 6534178]
[12301870, 7144136, 8144459, 1039855, 5564922]
False
flags= 0
size_len= 8660
[999999, 1082185, 1029743, 1098066, 6534178]
[10311335, 17330255, 1091793, 15511873, 1105381]
False
flags= 0
size_len= 8665
[999999, 1082185, 1127831, 995785, 1068719]
[13411817, 931432, 837694, 1353499, 898513]
False
flags= 0
size_len= 8670
[999999, 1082185, 1029743, 1098066, 6534178]
[190498, 1047959, 959891, 1011319, 6548394]
False
flags= 0
size_len= 8675
[999999]
[874865, 8358046, 1134678, 624757, 864143]
False
flags= 0
size_len= 8680
[999999, 1082185, 1098066, 6534178, 1127831]
[9835630, 1106002, 3868624, 947427, 830062]
False
flags= 0
size_len= 8685
[999999, 1082185, 6534178, 1044078, 1133018]
[8181028, 9827666, 9337203, 12524124, 861038]
False
flags= 0
size_len= 8690
[999999, 1082185, 1029743, 1098066, 1127831]
[15452762, 51

size_len= 9660
[999999, 1029743, 1098066, 1068719, 1053690]
[5690475, 9858981, 925483, 15452577, 867548]
False
flags= 0
size_len= 9665
[999999, 1082185, 1029743, 995785, 1033142]
[10204672, 262171, 13672246, 10457520, 2232740]
False
flags= 0
size_len= 9670
[999999]
[1034138, 1488652, 1926712, 10142345, 10452861]
False
flags= 0
size_len= 9675
[999999, 1082185, 1098066, 6534178, 1127831]
[1926651, 15630027, 9879244, 2540205, 973476]
False
flags= 0
size_len= 9680
[999999, 1082185, 1029743, 1098066, 6534178]
[13118251, 9297299, 9796167, 45077, 10284898]
False
flags= 0
size_len= 9685
[999999, 1029743, 1044078, 1096036, 1053690]
[1302756, 13877270, 957874, 1038038, 927992]
False
flags= 0
size_len= 9690
[999999, 1082185, 1029743, 1098066, 6534178]
[9850169, 939602, 15598951, 435602, 1190388]
False
flags= 0
size_len= 9695
[999999, 1082185, 1098066, 6534178, 995785]
[1786824, 962935, 902192, 963632, 15778485]
False
flags= 0
size_len= 9700
[999999, 1082185, 1127831, 1081177, 1133018]
[786378, 10

In [182]:
# random_recommendation
print(precision_at_k(result['popular_recommendation'], result['own_purchases'], k=5))

2042
False
flags= 0
size_len= 5
False
flags= 0
size_len= 10
False
flags= 0
size_len= 15
False
flags= 0
size_len= 20
False
flags= 0
size_len= 25
False
flags= 0
size_len= 30
False
flags= 0
size_len= 35
False
flags= 0
size_len= 40
False
flags= 0
size_len= 45
False
flags= 0
size_len= 50
False
flags= 0
size_len= 55
False
flags= 0
size_len= 60
False
flags= 0
size_len= 65
False
flags= 0
size_len= 70
False
flags= 0
size_len= 75
False
flags= 0
size_len= 80
False
flags= 0
size_len= 85
False
flags= 0
size_len= 90
False
flags= 0
size_len= 95
False
flags= 0
size_len= 100
False
flags= 0
size_len= 105
False
flags= 0
size_len= 110
False
flags= 0
size_len= 115
False
flags= 0
size_len= 120
False
flags= 0
size_len= 125
False
flags= 0
size_len= 130
False
flags= 0
size_len= 135
False
flags= 0
size_len= 140
False
flags= 0
size_len= 145
False
flags= 0
size_len= 150
False
flags= 0
size_len= 155
False
flags= 0
size_len= 160
False
flags= 0
size_len= 165
False
flags= 0
size_len= 170
False
flags= 0
size_len= 175


flags= 0
size_len= 1750
False
flags= 0
size_len= 1755
False
flags= 0
size_len= 1760
False
flags= 0
size_len= 1765
False
flags= 0
size_len= 1770
False
flags= 0
size_len= 1775
False
flags= 0
size_len= 1780
False
flags= 0
size_len= 1785
False
flags= 0
size_len= 1790
False
flags= 0
size_len= 1795
False
flags= 0
size_len= 1800
False
flags= 0
size_len= 1805
False
flags= 0
size_len= 1810
False
flags= 0
size_len= 1815
False
flags= 0
size_len= 1820
False
flags= 0
size_len= 1825
False
flags= 0
size_len= 1830
False
flags= 0
size_len= 1835
False
flags= 0
size_len= 1840
False
flags= 0
size_len= 1845
False
flags= 0
size_len= 1850
False
flags= 0
size_len= 1855
False
flags= 0
size_len= 1860
False
flags= 0
size_len= 1865
False
flags= 0
size_len= 1870
False
flags= 0
size_len= 1875
False
flags= 0
size_len= 1880
False
flags= 0
size_len= 1885
False
flags= 0
size_len= 1890
False
flags= 0
size_len= 1895
False
flags= 0
size_len= 1900
False
flags= 0
size_len= 1905
False
flags= 0
size_len= 1910
False
flags= 0
s

flags= 0
size_len= 3250
False
flags= 0
size_len= 3255
False
flags= 0
size_len= 3260
False
flags= 0
size_len= 3265
False
flags= 0
size_len= 3270
False
flags= 0
size_len= 3275
False
flags= 0
size_len= 3280
False
flags= 0
size_len= 3285
False
flags= 0
size_len= 3290
False
flags= 0
size_len= 3295
False
flags= 0
size_len= 3300
False
flags= 0
size_len= 3305
False
flags= 0
size_len= 3310
False
flags= 0
size_len= 3315
False
flags= 0
size_len= 3320
False
flags= 0
size_len= 3325
False
flags= 0
size_len= 3330
False
flags= 0
size_len= 3335
False
flags= 0
size_len= 3340
False
flags= 0
size_len= 3345
False
flags= 0
size_len= 3350
False
flags= 0
size_len= 3355
False
flags= 0
size_len= 3360
False
flags= 0
size_len= 3365
False
flags= 0
size_len= 3370
False
flags= 0
size_len= 3375
False
flags= 0
size_len= 3380
False
flags= 0
size_len= 3385
False
flags= 0
size_len= 3390
False
flags= 0
size_len= 3395
False
flags= 0
size_len= 3400
False
flags= 0
size_len= 3405
False
flags= 0
size_len= 3410
False
flags= 0
s

False
flags= 0
size_len= 4750
False
flags= 0
size_len= 4755
False
flags= 0
size_len= 4760
False
flags= 0
size_len= 4765
False
flags= 0
size_len= 4770
False
flags= 0
size_len= 4775
False
flags= 0
size_len= 4780
False
flags= 0
size_len= 4785
False
flags= 0
size_len= 4790
False
flags= 0
size_len= 4795
False
flags= 0
size_len= 4800
False
flags= 0
size_len= 4805
False
flags= 0
size_len= 4810
False
flags= 0
size_len= 4815
False
flags= 0
size_len= 4820
False
flags= 0
size_len= 4825
False
flags= 0
size_len= 4830
False
flags= 0
size_len= 4835
False
flags= 0
size_len= 4840
False
flags= 0
size_len= 4845
False
flags= 0
size_len= 4850
False
flags= 0
size_len= 4855
False
flags= 0
size_len= 4860
False
flags= 0
size_len= 4865
False
flags= 0
size_len= 4870
False
flags= 0
size_len= 4875
False
flags= 0
size_len= 4880
False
flags= 0
size_len= 4885
False
flags= 0
size_len= 4890
False
flags= 0
size_len= 4895
False
flags= 0
size_len= 4900
False
flags= 0
size_len= 4905
False
flags= 0
size_len= 4910
False
flag

False
flags= 0
size_len= 6465
False
flags= 0
size_len= 6470
False
flags= 0
size_len= 6475
False
flags= 0
size_len= 6480
False
flags= 0
size_len= 6485
False
flags= 0
size_len= 6490
False
flags= 0
size_len= 6495
False
flags= 0
size_len= 6500
False
flags= 0
size_len= 6505
False
flags= 0
size_len= 6510
False
flags= 0
size_len= 6515
False
flags= 0
size_len= 6520
False
flags= 0
size_len= 6525
False
flags= 0
size_len= 6530
False
flags= 0
size_len= 6535
False
flags= 0
size_len= 6540
False
flags= 0
size_len= 6545
False
flags= 0
size_len= 6550
False
flags= 0
size_len= 6555
False
flags= 0
size_len= 6560
False
flags= 0
size_len= 6565
False
flags= 0
size_len= 6570
False
flags= 0
size_len= 6575
False
flags= 0
size_len= 6580
False
flags= 0
size_len= 6585
False
flags= 0
size_len= 6590
False
flags= 0
size_len= 6595
False
flags= 0
size_len= 6600
False
flags= 0
size_len= 6605
False
flags= 0
size_len= 6610
False
flags= 0
size_len= 6615
False
flags= 0
size_len= 6620
False
flags= 0
size_len= 6625
False
flag

size_len= 9245
False
flags= 0
size_len= 9250
False
flags= 0
size_len= 9255
False
flags= 0
size_len= 9260
False
flags= 0
size_len= 9265
False
flags= 0
size_len= 9270
False
flags= 0
size_len= 9275
False
flags= 0
size_len= 9280
False
flags= 0
size_len= 9285
False
flags= 0
size_len= 9290
False
flags= 0
size_len= 9295
False
flags= 0
size_len= 9300
False
flags= 0
size_len= 9305
False
flags= 0
size_len= 9310
False
flags= 0
size_len= 9315
False
flags= 0
size_len= 9320
False
flags= 0
size_len= 9325
False
flags= 0
size_len= 9330
False
flags= 0
size_len= 9335
False
flags= 0
size_len= 9340
False
flags= 0
size_len= 9345
False
flags= 0
size_len= 9350
False
flags= 0
size_len= 9355
False
flags= 0
size_len= 9360
False
flags= 0
size_len= 9365
False
flags= 0
size_len= 9370
False
flags= 0
size_len= 9375
False
flags= 0
size_len= 9380
False
flags= 0
size_len= 9385
False
flags= 0
size_len= 9390
False
flags= 0
size_len= 9395
False
flags= 0
size_len= 9400
False
flags= 0
size_len= 9405
False
flags= 0
size_len= 

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code